## Week 6 notebook

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r'/Users/Jvon/Masters/2025-summer-mod-6/homework_6.1.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Z,X,Y
0,0.548814,0,-0.823220
1,0.715189,1,0.842405
2,0.602763,1,0.898618
3,0.544883,0,-0.817325
4,0.423655,0,-0.635482


### Matching and Treatment Effect Analysis

- Match treated (X=1) to untreated (X=0) based on confounder (Z) using NearestNeighbors.
- Compute the average treatment effect (ATE) by pairing each item with its nearest counterfactual.
- Compute the average treatment effect on the treated (ATT).
- Compute the average treatment effect on the untreated (ATU).
- Compute the marginal treatment effect (maximum effect among untreated).

2. Propose a solution that remedies this problem and write some code that implements your solution.  It's very important here that you clearly explain what your solution will do.
Possible answer: maybe we could take the 90th percentile of the treatment effect and use it as a proxy for the Marginal Treatment Effect.
(Either code this answer or choose a different one.)


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Assume df has columns: X (treatment), Y (outcome), Z (confounder)
# Separate treated and untreated
T = df[df['X'] == 1].copy()
U = df[df['X'] == 0].copy()

# Fit NearestNeighbors on Z for each group
t_nn = NearestNeighbors(n_neighbors=1).fit(U[['Z']])
u_nn = NearestNeighbors(n_neighbors=1).fit(T[['Z']])

# For each treated, find nearest untreated (counterfactual)
dist_T, idx_T = t_nn.kneighbors(T[['Z']])
T['cf_idx'] = idx_T.flatten()
T['cf_Y'] = U.iloc[T['cf_idx']]['Y'].values
T['effect'] = T['Y'] - T['cf_Y']

# For each untreated, find nearest treated (counterfactual)
dist_U, idx_U = u_nn.kneighbors(U[['Z']])
U['cf_idx'] = idx_U.flatten()
U['cf_Y'] = T.iloc[U['cf_idx']]['Y'].values
U['effect'] = U['cf_Y'] - U['Y']

# Average Treatment Effect (ATE): average of all effects (both directions)
ate = pd.concat([T['effect'], U['effect']]).mean()

# Average Treatment Effect on the Treated (ATT): average effect for treated
att = T['effect'].mean()

# Average Treatment Effect on the Untreated (ATU): average effect for untreated
atu = U['effect'].mean()

# Marginal Treatment Effect: maximum effect among untreated and the 90th percentile among untreated
mte = U['effect'].max()
mte_90th_percent = U['effect'].quantile(0.9)

print(f"ATE: {ate:.3f}")
print(f"ATT: {att:.3f}")
print(f"ATU: {atu:.3f}")
print(f"Marginal Treatment Effect: {mte:.3f}")
print(f"90th Percentile of Marginal Treatment Effect: {mte_90th_percent:.3f}")

ATE: 1.695
ATT: 1.846
ATU: 1.549
Marginal Treatment Effect: 2.172
90th Percentile of Marginal Treatment Effect: 1.928


1. What is a potential problem with computing the Marginal Treatment Effect simply by comparing each untreated item to its counterfactual and taking the maximum difference?  (Hint: think of statistics here.  Consider that only the most extreme item ends up being used to estimate the MTE.  That's not necessarily a bad thing; the MTE is supposed to come from the untreated item that will produce the maximum effect.  But there is nevertheless a problem.)
Possible answer: We are likely to find the item with the most extreme difference, which may be high simply due to randomness.
(Please explain / justify this answer, or give a different one if you can think of one.)

The problem is that by only considering the maximum difference, we may be ignoring other untreated items that could provide valuable information about the treatment effect. This could lead to an overestimation of the MTE if the maximum difference is an outlier.